# Яндекс.Музыка

**Цель исследования** — проверьте три гипотезы:
1. Активность пользователей зависит от дня недели. Причём в Москве и Петербурге это проявляется по-разному.
2. В понедельник утром в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города. 
3. Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.

**Ход исследования**
 1. Обзор данных.
 2. Предобработка данных.
 3. Проверка гипотез.

## Обзор данных


In [1]:
import pandas as pd # импорт библиотеки pandas

In [2]:
df = pd.read_csv('yandex_music_project.csv') # чтение файла с данными и сохранение в df

In [3]:
df.head(10)  #получение первых 10 строк таблицы df)

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,NaN,ruspop,Moscow,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Saint-Petersburg,21:20:49,Wednesday


In [4]:
df.info()# получение общей информации о данных в таблице df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


Итак, в таблице семь столбцов. Тип данных во всех столбцах — `object`.

Согласно документации к данным:
* `userID` — идентификатор пользователя;
* `Track` — название трека;  
* `artist` — имя исполнителя;
* `genre` — название жанра;
* `City` — город пользователя;
* `time` — время начала прослушивания;
* `Day` — день недели.

**Выводы**

Каждая строка таблицы содержит информацию о композициях определённого жанра в определённом исполнении, которые пользователи слушали в одном из городов в определённое время и день недели. Две проблемы, которые нужно решать: пропуски и некачественные названия столбцов. Для проверки рабочих гипотез особенно ценны столбцы *time*, *day* и *City*. Данные из столбца *genre* позволят узнать самые популярные жанры.

## Предобработка данных


### Переименование столбцов

In [5]:
df.columns # перечень названий столбцов таблицы df

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

В названиях столбцов есть пробелы, которые могут затруднять доступ к данным.

Переименуем столбцы для удобства дальнейшей работы. Проверим результат.

In [40]:
new_names=['user_id','track_name','artist_name', 'genre_name', 'city','time', 'weekday']
df.set_axis(new_names, axis='columns', inplace=True)

Проверьте результат. Для этого ещё раз выведите на экран названия столбцов:

In [7]:
df.columns # проверка результатов - перечень названий столбцов

Index(['user_id', 'track_name', 'artist_name', 'genre_name', 'city', 'time',
       'weekday'],
      dtype='object')

### Пропуски значений
Сначала посчитайте, сколько в таблице пропущенных значений. Для этого достаточно двух методов `pandas`:

In [8]:
 df.isna().sum() # подсчёт пропусков

user_id           0
track_name     1231
artist_name    7203
genre_name     1198
city              0
time              0
weekday           0
dtype: int64

Пустые значения свидетельствуют, что для некоторых треков доступна не вся информация. Причины могут быть разные: скажем, не назван конкретный исполнитель народной песни. Хуже, если проблемы с записью данных. Каждый отдельный случай необходимо разобрать и выявить причину.


Заменяем пропущенные значения в столбцах с названием трека и исполнителя на строку 'unknown'. После этой операции нужно убедиться, что таблица больше не содержит пропусков.

In [9]:
df['track_name']=df['track_name'].fillna('unknown')# <замена пропущенных значений в столбце 'track_name' на строку 'unknown' специальным методом замены>
df.isnull().sum()

user_id           0
track_name        0
artist_name    7203
genre_name     1198
city              0
time              0
weekday           0
dtype: int64

In [10]:
# <замена пропущенных значений в столбце 'artist_name' на строку 'unknown' специальным методом замены>
df['artist_name']=df['artist_name'].fillna('unknown')

Удаляем в столбце с жанрами пустые значения; убеждаемся, что их больше не осталось.

In [11]:
# <удаление пропущенных значений в столбце 'genre_name'>
df.dropna(subset=['genre_name'],inplace=True)

Убедимся, что в таблице не осталось пропусков. Для этого ещё раз посчитайте пропущенные значения.

In [12]:
 df.isna().sum() # подсчёт пропусков

user_id        0
track_name     0
artist_name    0
genre_name     0
city           0
time           0
weekday        0
dtype: int64

### Дубликаты
Необходимо установить наличие дубликатов.  Если найдутся, удаляем, и проверяем, все ли удалились.

In [13]:
df.duplicated().sum() # подсчёт явных дубликатов

3755

In [14]:
# удаление явных дубликатов (с удалением старых индексов и формированием новых)
df = df.drop_duplicates().reset_index(drop=True) 

In [15]:
df.duplicated().sum() # проверка на отсутствие дубликатов

0

Теперь избавьтесь от неявных дубликатов в колонке `genre`. Например, название одного и того же жанра может быть записано немного по-разному. Такие ошибки тоже повлияют на результат исследования.

Сохраняем список уникальных значений столбца с жанрами в переменной genres_list.

Объявим функцию find_genre() для поиска неявных дубликатов в столбце с жанрами. Например, когда название одного и того же жанра написано разными словами.

In [16]:
df['genre_name'].sort_values().unique()   # Просмотр уникальных названий жанров

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

Мы видим следующие неявные дубликаты:
* *hip*,
* *hop*,
* *hip-hop*.

Объявим функцию *replace_wrong_genres()*, которая заменяет  неправильное название этого жанра в столбце *'genre_name'* на *'hiphop'* и проверяет успешность выполнения замены.

Так исправляем все варианты написания, которые выявила проверка.

In [17]:
def replace_wrong_genres(wrong_genres,correct_genre):# Функция для замены неявных дубликатов
    df['genre_name']= df['genre_name'].replace(wrong_genres, correct_genre)

In [18]:
replace_wrong_genres(['hip', 'hop', 'hip-hop'], 'hiphop') # Устранение неявных дубликатов

Проверим, что заменили неправильные названия:

*   hip
*   hop
*   hip-hop

Выведите отсортированный список уникальных значений столбца `genre`:

In [19]:
df['genre_name'].sort_values().unique() # Проверка на неявные дубликаты

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

In [20]:
# <получение общей информации о данных таблицы df>
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60126 entries, 0 to 60125
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      60126 non-null  object
 1   track_name   60126 non-null  object
 2   artist_name  60126 non-null  object
 3   genre_name   60126 non-null  object
 4   city         60126 non-null  object
 5   time         60126 non-null  object
 6   weekday      60126 non-null  object
dtypes: object(7)
memory usage: 3.2+ MB


**Выводы**
На этапе предобработки в данных обнаружились не только пропуски и проблемы с названиями столбцов, но и   дубликаты. Их удаление позволит провести анализ точнее. Поскольку сведения о жанрах важно сохранить для анализа, не просто удаляем все пропущенные значения, но заполним пропущенные имена исполнителей и названия треков. Имена столбцов теперь корректны и удобны для дальнейшей работы.

## Проверка гипотез

### Сравнение поведения пользователей двух столиц

Была выдвинута гипотеза, что в Москве и Санкт-Петербурге пользователи слушают музыку по-разному. Проверяем это предположение по данным о трёх днях недели — понедельнике, среде и пятнице.

Для каждого города устанавливаем количество прослушанных  в эти дни композиций с известным жанром, и сравниваем результаты.

In [21]:
# <группировка данных таблицы df по столбцу 'city' и подсчёт количества значений столбца 'genre_name'>
df.groupby('city')['genre_name'].count()

city
Moscow              41892
Saint-Petersburg    18234
Name: genre_name, dtype: int64

В Москве прослушиваний больше, чем в Петербурге. Из этого не следует, что московские пользователи чаще слушают музыку. Просто самих пользователей в Москве больше.

Сгруппируем данные по дню недели и подсчитаем прослушанные в понедельник, среду и пятницу композиции, для которых известен жанр.

In [22]:
# <группировка данных по столбцу 'weekday' и подсчёт количества значений столбца 'genre_name'>
df.groupby('weekday')['genre_name'].count()

weekday
Friday       21482
Monday       20866
Wednesday    17778
Name: genre_name, dtype: int64

Понедельник и пятница — время для музыки; по средам пользователи немного больше вовлечены в работу.

Создаём функцию *number_tracks()*, которая принимает как параметры таблицу, день недели и название города, а возвращает количество прослушанных композиций, для которых известен жанр. Проверяем количество прослушанных композиций для каждого города и понедельника, затем среды и пятницы.

In [23]:
df_moscow = df.loc[df['city']=='Moscow']
df_mon_msk = df_moscow.loc[df['weekday']=='Monday']['user_id'].count()
df_wed_msk = df_moscow.loc[df['weekday']=='Wednesday']['user_id'].count()
df_fri_msk = df_moscow.loc[df['weekday']=='Friday']['user_id'].count()
print(f'Прослушиваний по дням в Москве:{df_mon_msk} - понедельник, {df_wed_msk} - среда, {df_fri_msk}- пятница')

df_piter = df.loc[df['city']=='Saint-Petersburg']
df_mon_pit = df_piter.loc[df['weekday']=='Monday']['user_id'].count()
df_wed_pit = df_piter.loc[df['weekday']=='Wednesday']['user_id'].count()
df_fri_pit = df_piter.loc[df['weekday']=='Friday']['user_id'].count()
print(f'Прослушиваний по дням в Санкт-Петербурге:{df_mon_pit} - понедельник, {df_wed_pit} - среда, {df_fri_pit}- пятница')

Прослушиваний по дням в Москве:15347 - понедельник, 10865 - среда, 15680- пятница
Прослушиваний по дням в Санкт-Петербурге:5519 - понедельник, 6913 - среда, 5802- пятница


Сведём полученную информацию в одну таблицу, где ['city', 'monday', 'wednesday', 'friday'] названия столбцов.


In [24]:
data = [['Moscow', df_mon_msk, df_wed_msk, df_fri_msk],
       ['Saint-Petersburg', df_mon_pit, df_wed_pit,df_fri_pit]]
columns = ['city', 'monday', 'wednesday', 'friday']
research_result = pd.DataFrame(data=data,columns=columns) # Таблица с результатами
print(research_result)

               city  monday  wednesday  friday
0            Moscow   15347      10865   15680
1  Saint-Petersburg    5519       6913    5802


**Выводы**
Результаты показывают, что относительно среды музыку в Петербурге и Москве слушают «зеркально»: в Москве пики приходятся на понедельник и пятницу, а в среду время прослушивания снижается. Тогда как в Санкт-Петербурге среда — день самого большого интереса к музыке, а в понедельник и пятницу он меньше, причём почти одинаково меньше.

Значит, данные говорят в пользу гипотезы.

### Музыка в начале и в конце недели

Согласно второй гипотезе, утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

Создадим 2 таблицы с данными в две переменные:
* по Москве — в `moscow_general`;
* по Санкт-Петербургу — в `spb_general`.

In [25]:
moscow_general=df[df['city']=='Moscow']

In [26]:
spb_general=df[df['city']=='Saint-Petersburg']

Создаём функцию *genre_weekday()*, которая возвращает список жанров по запрошенному дню недели и времени суток с такого-то часа по такой-то.

In [30]:
# объявление функции genre_weekday() с параметрами df, day, time1, time2
# в переменной genre_list сохраняются те строки df, для которых одновременно:
# 1) значение в столбце 'weekday' равно параметру day,
# 2) значение в столбце 'time' больше time1 и
# 3) меньше time2.
# в переменной genre_list_sorted сохраняются в порядке убывания  
# первые 10 значений Series, полученной подсчётом числа значений 'genre_name'
# сгруппированной по столбцу 'genre_name' таблицы genre_list
# функция возвращает значение genre_list_sorted
def genre_weekday(df, day, time1, time2):
    genre_list = df[(df['weekday'] == day) & (df['time'] > time1) & (df['time'] < time2)]
    genre_list_sorted=genre_list.sort_values(by = 'genre_name', ascending = False).head(10)
    return genre_list_sorted

Cравниваем полученные результаты по таблице для Москвы и Санкт-Петербурга в понедельник утром (с 7 до 11) и в пятницу вечером (с 17 до 23).

In [31]:
# <вызов функции для утра понедельника в Москве (вместо df таблица moscow_general)>
genre_weekday(moscow_general, 'Monday', '07:00:00', '11:00:00')

,user_id,track_name,artist_name,genre_name,city,time,weekday
27442,D7FB50DA,Drumming Circle,Professor Trance,worldbeat,Moscow,09:30:47,Monday
20384,F0C5DA9A,Hasta Siempre Comandante,Juan M. Chazarreta,world,Moscow,08:10:09,Monday
36691,C3B36125,Brava la cuoca,Roberto Scaglioni,world,Moscow,09:51:28,Monday
54812,F37EC6D1,Penjereden Dash Gelir,Chingiz Sadykhov,world,Moscow,08:29:31,Monday
58660,59CBCCE5,Dyolmano Dyulbero,The Bulgarian Voices,world,Moscow,09:53:19,Monday
7388,62E1B964,Child of a Blind Man,Hazmat Modine,world,Moscow,09:37:22,Monday
21143,FF3FD2BD,Ire,Adekunle Gold,world,Moscow,08:26:19,Monday
49652,FE3A4C70,Ма царды 'мбал (Спутник жизни),Оксана Джелиева,world,Moscow,08:53:25,Monday
26801,FDF6D919,Nuage,Jim Cole & Spectral Voices,world,Moscow,08:42:04,Monday
40820,3D844B0C,Tango,Athens Symphony Orchestra,world,Moscow,09:45:53,Monday


In [32]:
# <вызов функции для утра понедельника в Петербурге (вместо df таблица spb_general)>
genre_weekday(spb_general, 'Monday', '07:00:00', '11:00:00')

,user_id,track_name,artist_name,genre_name,city,time,weekday
30068,692AEC35,Baiana,Emicida,world,Saint-Petersburg,09:24:38,Monday
42316,6B62E9CE,The Color of The Night,L' Amore Orchestra,world,Saint-Petersburg,08:56:04,Monday
25542,F018BD36,Aicha,Khaled,world,Saint-Petersburg,09:24:54,Monday
25368,888CB0C0,Sichou Zhi Lu,Ethno Music Orchestra,world,Saint-Petersburg,08:49:11,Monday
44109,2DD633B3,Parole parole,unknown,world,Saint-Petersburg,08:50:08,Monday
45091,B5B6FE0A,Sade,Badmos,world,Saint-Petersburg,09:03:38,Monday
21206,F33FA8C,Pivni Tkach posh edit light,Katya Chilly,world,Saint-Petersburg,08:51:27,Monday
20813,4D2A996C,Closer to Heaven,Mysteria,world,Saint-Petersburg,09:30:23,Monday
19541,CA471C2,The Chieri by Avalon Rising,Avalon Rising,world,Saint-Petersburg,09:05:43,Monday
18608,FEA172A3,Jenny Chicken,Kate Mcmayland and Her Orchestra,world,Saint-Petersburg,09:04:32,Monday


In [33]:
# <вызов функции для вечера пятницы в Москве>
genre_weekday(moscow_general, 'Friday', '17:00:00', '23:00:00')

,user_id,track_name,artist_name,genre_name,city,time,weekday
43694,917047E8,The Nomad,Niraj Chag Featuring Melissa Baten,world,Moscow,21:58:43,Friday
30922,3DC9EF93,Aprum Em Keznov,Arabo Ispiryan,world,Moscow,20:07:26,Friday
31820,9F87ADDC,Bir Gozal,unknown,world,Moscow,20:14:37,Friday
52603,4C61D12D,Apple Penne,Bharathvaj,world,Moscow,21:32:05,Friday
31514,5366E775,Die Reblaus,Hans Moser,world,Moscow,21:40:41,Friday
31317,CAAC4375,Kervansaray,Muhammet İrmikçi,world,Moscow,20:22:48,Friday
2878,2487DD6F,Макарий,Абстрактор,world,Moscow,20:57:45,Friday
14375,BCEA335,Yo Soy Maria,Panterinya,world,Moscow,21:11:20,Friday
31013,77BE1010,Kandıramazsın,Genco Ecer,world,Moscow,22:00:25,Friday
52657,6214F479,Forró do Rei,Trio Virgulino,world,Moscow,20:28:05,Friday


In [34]:
# <вызов функции для вечера пятницы в Питере>
genre_weekday(spb_general, 'Friday', '17:00:00', '23:00:00')

,user_id,track_name,artist_name,genre_name,city,time,weekday
54324,54A01EAC,Batwanes Beek Ta'a Ninsa,emad batayeh,world,Saint-Petersburg,20:58:18,Friday
29985,47E5088,Что же делать?,Группа «Домбай»,world,Saint-Petersburg,21:15:04,Friday
44531,873D8A1B,Orkestarsko Oro,Maleševski Melos,world,Saint-Petersburg,20:58:55,Friday
18013,1D507F3F,A Cochabamba Me Voy,Daniele Sepe,world,Saint-Petersburg,21:50:57,Friday
48238,F3FB8127,Dari Lolo,Aravod,world,Saint-Petersburg,21:48:19,Friday
22457,A38F488E,The Forest,Taiko Drums,world,Saint-Petersburg,21:27:03,Friday
3470,86F2380D,Королева,Мануэль Исаков,world,Saint-Petersburg,21:19:05,Friday
28527,807DDAF6,El Forga Moura,Haim,world,Saint-Petersburg,21:11:00,Friday
38883,BF27088D,Let It Out - Rafijah & Niyo Rah,Rafijah & Niyo Rah,world,Saint-Petersburg,21:28:56,Friday
29561,DAE710B3,Les nuits de patrimonio,Jérôme Ciosi,world,Saint-Petersburg,20:07:58,Friday


Популярные жанры в понедельник утром в Питере и Москве оказались похожи: везде, как и предполагалось, популярен поп. Несмотря на это, концовка топ-10 для двух городов различается: в Питере в топ-10 входит джаз и русский рэп, а в Москве жанр *world*.

В конце недели ситуация не меняется. Поп-музыка всё так же на первом месте. Опять разница заметна только в концовке топ-10, где в Питере пятничным вечером тоже присутствует жанр *world*.

**Выводы**

Жанр поп безусловный лидер, а топ-5 в целом не различается в обеих столицах. При этом видно, что концовка списка более «живая»: для каждого города выделяются более характерные жанры, которые действительно меняют свои позиции в зависимости от дня недели и времени.

### Жанровые предпочтения в Москве и Петербурге

Гипотеза: Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве.  А Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка.

Сгруппируем таблицу *moscow_general* по жанру, сосчитаем численность композиций каждого жанра методом *count()*, отсортируем в порядке убывания и сохраним результат в таблице *moscow_genres*.

Просмотрим первые 10 строк этой новой таблицы.

In [38]:
moscow_genres = moscow_general.groupby('genre_name')['genre_name'].count().sort_values(ascending = False)
moscow_genres.head(10)

genre_name
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre_name, dtype: int64

Сгруппируем таблицу *spb_general* по жанру, сосчитаем численность композиций каждого жанра методом *count()*, отсортируем в порядке убывания и сохраним результат в таблице *spb_genres*.

Просматриваем первые 10 строк этой таблицы. Теперь можно сравнивать два города.

In [39]:
# <группировка таблицы spb_general, расчёт, сохранение в spb_genres>
spb_genres = spb_general.groupby('genre_name')['genre_name'].count().sort_values(ascending = False)
spb_genres.head(10)

genre_name
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre_name, dtype: int64

**Выводы**

Гипотеза частично подтвердилась:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге. 


## Итоги исследования

Рабочие гипотезы:

* музыку в двух городах — Москве и Санкт-Петербурге — слушают в разном режиме;

* списки десяти самых популярных жанров утром в понедельник и вечером в пятницу имеют характерные отличия;

* население двух городов предпочитает cхожие музыкальные жанры.

Общие результаты

Москва и Петербург сходятся во вкусах: везде преобладает популярная музыка. При этом зависимости предпочтений от дня недели в каждом отдельном городе нет — люди постоянно слушают то, что им нравится. Но между городами в разрезе дней неделей наблюдается зеркальность относительно среды: Москва больше слушает в понедельник и пятницу, а Петербург наоборот - больше в среду, но меньше в понедельник и пятницу.
